In [1]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
import re
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
okt=Okt()

/Users/heeseok/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [13]:
import telepot
from telegram.bot import Bot

my_token = '900554116:AAGPWCrlk4QsIItA54KPR02MWNB2C7a-tCM'

bot = Bot(token = my_token)

updates = bot.getUpdates() 
for u in updates:
    print(u.message)

chat_id = bot.getUpdates()[-1].message.chat.id
chat_id

bot.sendMessage(chat_id = chat_id, text="안녕하세요")

{'message_id': 24, 'date': 1566544895, 'chat': {'id': 612130331, 'type': 'private', 'first_name': 'h', 'last_name': 'js'}, 'text': 'hi', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 612130331, 'first_name': 'h', 'is_bot': False, 'last_name': 'js', 'language_code': 'ko'}}
{'message_id': 33, 'date': 1566546097, 'chat': {'id': 612130331, 'type': 'private', 'first_name': 'h', 'last_name': 'js'}, 'text': 'ㄱㅅㄱㅅ', 'entities': [], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 612130331, 'first_name': 'h', 'is_bot': False, 'last_name': 'js', 'language_code': 'ko'}}
{'message_id': 34, 'date': 1566626291, 'chat': {'id': 612130331, 'type':

In [5]:
def searchResult(_Q):
    
    searchResult = defaultdict(int)

    with open('new_postings.txt') as Postings_:
        for t in _Q:

            j = V.index(t)
            nextPosition = Dictionary[j]
            while True:
                Postings_.seek(nextPosition)
                _struct = [int(_.strip()) for _ in Postings_.readline().split(",")]
                #_struct[1] => 몇번나왔니
                searchResult[_struct[0]] +=1
                nextPosition = _struct[2]

                if nextPosition == -1:

                    break
        totalResult = sorted(searchResult.items(), key=lambda x:x[1], reverse=True)
    
    return totalResult

In [6]:
def select_result(totalResult):
    mx = max([_[1] for _ in totalResult])
    
    index_list=[]
    
    while sum([len(_) for _ in index_list])<=50 and mx>0:
        
        index_list.append([_[0] for _ in totalResult if _[1]==mx])
        mx -=1
        
    return index_list

In [7]:
def minwon_search(Q,data):
    _Q = [_[0] for _ in okt.pos(Q) if _[1].startswith("N") and _[0] in V]
    totalResult = searchResult(_Q)
    _Q = ' '.join(_Q)
    print(_Q)
    index_result = select_result(totalResult)
    
    ##########---------------------------------------#########
    #      여기까지하면 index_result[i] :mx-i개 키워드 포함한 doc_id#
    ##########---------------------------------------#########
    
    
    rankedIdx=[]
    for i in range(len(index_result)):
        #######-----------------------------##################
        #     db에서 불러와야하면                                #
        # index_result[i]에 있는 doc_id들로 불러와서             #
        # dataFrame을 만들어야한다                             #
        #######-----------------------------################
        temp = data.controlled_collection[index_result[i]]
        temp = np.append(temp,_Q)
        tfidf = TfidfVectorizer(vocabulary=V,sublinear_tf=True)
        tf_idf_matrix = tfidf.fit_transform(temp)
        cosine_sim = linear_kernel(tf_idf_matrix,tf_idf_matrix)
        cosine_sim = sorted(cosine_sim, key=lambda x:x[-1], reverse=True)[1:51]
        for _ in cosine_sim:
            idx = np.where(_==max(_))[0][0]
            rankedIdx.append(index_result[i][idx])

    return rankedIdx

데이터 읽어오기

In [2]:
data = pd.read_csv('new_mashed_up.csv')
len(data)

/Users/heeseok/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


116508

lexicon 읽어오기

In [3]:
termLex = pd.read_csv('new_dictionary.csv')

Dictionary = defaultdict(lambda:-1)
for idx,pos in enumerate(termLex.dictionary):
    Dictionary[idx]=pos
    
V = termLex.vocabulary.to_list()

In [8]:
result = rankedIdx = minwon_search('한남동 폐차',data)
print(result)


한남동 폐차
[105351, 18532, 102267, 103922, 71937, 79002, 76219, 79877, 98560, 1543, 81007, 1517, 105606, 71939, 78750, 77963, 36775, 44076, 70155, 69598, 86321, 99931, 82335, 57817, 79658, 43501, 43501, 106874, 105405, 32521, 75074, 44965, 66147, 110855, 43511, 43511, 58062, 40303, 79976, 36813, 68875, 46083, 89435, 1541, 94985, 102255, 100786, 9446, 93981, 40427, 1010]


In [20]:
for _ in rankedIdx[:5]:
    print(re.sub(r"\r",'\n',data.iloc[_].title.strip()),'   조회수:',int(data.iloc[_]['view']))
    print('\n')
    print('---------------------------------------')
    print(re.sub(r"\r|<br\/>",'\n',data.iloc[_].question.strip()))
    print('\n')
    print('---------------------------------------')
    print(re.sub(r"\r|<br\/>",'\n',data.iloc[_].ans.strip()))
    print('=======================================')

폐기 차량 처분 요청 및 불법 주차 단속    조회수: 113


---------------------------------------
한남동 732-13 우진 주택앞, 이슬람서원 입구 우측 골목에 주인없이 폐차가 방치 되어 있는데 처리를 해주셨으면 합니다.(번호 : 61버 2046)
 그리고 주택 앞에 구청에거 발급한 주차금지 표지가 있는데도 불구하고 여전히 불법주차를 하고 있는데
단속을 해주셨으면 합니다. 
 마지막으로 이슬람서원은 많은 관광객들이 찾아오는 곳입니다. 주변의 청소가 요즘 거의 이루어지고 있지 않습니다. 
 주민들의 보다 쾌적하고 관광객들에게 한국의 좋은 이미지를 심어주었으면 합니다.
 감사합니다


---------------------------------------
안녕하십니까? 우리구의 발전을 위해 항상 협조하여 주시는 귀하께 감사드리며, 귀하께서 “불법주정차 단속과 관련하여 민원신청”하신 사항에 대해 답변 드리겠습니다.   [답변내용] 귀하께서 요청하신 한남동 732-13 우진 주택 주변 상습 불법주차 차량에 대하여 주기적인 순회단속을 실시하여 불법주정차 근절을 위해 최선을 다할 것임을 알려드립니다. 불법주정차로 불편을 겪으실 경우 평일주간(09:00~18:00)에는 교통지도과(☎2199-7800)로 연락주시기 바라며 야간 및 토,일(공휴일)에는 또는 용산구청 당직실(☎2199-6300)로 연락주시면 현장 출동하여 신속히 처리토록 하겠습니다. 또한, 우리구는 PDA시스템을 이용한 민원처리 전담 단속차량 4대가 교대로 단속 민원을 처리하고 있음을 알려드립니다. 우리구 주차질서 확립을 위하여 건의 해주신 귀하께 감사드리며, 귀하의 가정에 건강과 행운이 함께 하시길 기원합니다. 감사합니다.                           2011. 1. 21.    교통지도과(02-2199-7800) 담당: 한광수 / 팀장: 설동한 / 과장: 윤두용---------------------------------------------------

# Posting 만들기

In [45]:
from collections import defaultdict

V = list()
D = list()
Dictionary = defaultdict(lambda:-1)

Postings = open('./new_postings.txt', 'w')



for d in collection[:10000]:

    i = len(D)
    D.append(i)
    localPostings = defaultdict(int)
    

    for t in [_[0] for _ in okt.pos(d) if _[1].startswith("N") and len(_[0])>1]:
        if t not in V:
            V.append(t)
        j = V.index(t)
        localPostings[j] += 1
        
    for j, f in localPostings.items():
        nextPosition = Dictionary[j]
        _struct = (i, f, nextPosition)
        Dictionary[j] = Postings.tell()
        Postings.write(str(_struct[0])+','+str(_struct[1])+','+str(_struct[2])+'\n')
        
    if len(D)%50 ==0:
        print(len(D))
    
Postings.close()  

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [12]:
collection = data.collection

In [15]:
termLex = pd.read_csv('new_dictionary.csv')

Dictionary = defaultdict(lambda:-1)
for idx,pos in enumerate(termLex.dictionary):
    Dictionary[idx]=pos
    
V = termLex.vocabulary.to_list()
D = np.load('new_saved_doc_length.npy').tolist()

with open('./new_postings.txt', 'a') as Postings:
    for d in collection[len(D):]:

        i = len(D)
        D.append(i)
        localPostings = defaultdict(int)


        for t in [_[0] for _ in okt.pos(d) if _[1].startswith("N") and len(_[0])>1]:
            if t not in V:
                V.append(t)
            j = V.index(t)
            localPostings[j] += 1

        for j, f in localPostings.items():
            nextPosition = Dictionary[j]
            _struct = (i, f, nextPosition)
            Dictionary[j] = Postings.tell()
            Postings.write(str(_struct[0])+','+str(_struct[1])+','+str(_struct[2])+'\n')
        if len(D)%50 ==0:
            print("Vocabsize : ",len(V),"number of Docs : ", len(D))
            bot.sendMessage(chat_id = chat_id, text="Vocabsize : "+str(len(V))+"number of Docs : "+ str(len(D)))
        
        

termLex= pd.DataFrame()
termLex["vocabulary"] = V
dict_ = [Dictionary[_] for _ in range(len(V))]
termLex["dictionary"] = dict_
termLex.to_csv('new_dictionary.csv',header=True, index=False)
np.save('new_saved_doc_length',np.array(D))
print("termLex saved: length=",len(termLex),' saved_doc_length : ',len(D))

Vocabsize :  143598 number of Docs :  114850
Vocabsize :  143648 number of Docs :  114900
Vocabsize :  143687 number of Docs :  114950
Vocabsize :  143737 number of Docs :  115000
Vocabsize :  143784 number of Docs :  115050
Vocabsize :  143820 number of Docs :  115100
Vocabsize :  143863 number of Docs :  115150
Vocabsize :  143906 number of Docs :  115200
Vocabsize :  143945 number of Docs :  115250
Vocabsize :  143983 number of Docs :  115300
Vocabsize :  144020 number of Docs :  115350
Vocabsize :  144058 number of Docs :  115400
Vocabsize :  144097 number of Docs :  115450
Vocabsize :  144138 number of Docs :  115500
Vocabsize :  144172 number of Docs :  115550
Vocabsize :  144196 number of Docs :  115600
Vocabsize :  144215 number of Docs :  115650
Vocabsize :  144247 number of Docs :  115700
Vocabsize :  144265 number of Docs :  115750
Vocabsize :  144276 number of Docs :  115800
Vocabsize :  144301 number of Docs :  115850
Vocabsize :  144333 number of Docs :  115900
Vocabsize 

# controlled collection 만들기

In [24]:
temp = []
##바꾸셔야할부분
for d in data.collection:
    ctr_d = [_[0] for _ in okt.pos(d) if _[1].startswith("N") and len(_[0])>1]
    temp.append(' '.join(ctr_d))
    if(len(temp)%50 == 0):
        print(len(temp))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650


In [ ]:
data.controlled_collection = temp

In [31]:
data.to_csv('new_mashed_up.csv',header=True, index=False)